In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

In [2]:
# <span style="color:red">ch1_허깅페이스</span>
- Inference API 이용 : 모델의 결과를 sur
- 

SyntaxError: invalid syntax (1649882957.py, line 2)

In [4]:
import warnings
import os
import logging
# 경고 제거
warnings.filterwarnings('ignore')

# transformers 로깅 레벨 조정
logging.getLogger("transformers").setLevel(logging.ERROR)

# Hugging Face symlink 경고 제거
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# from transformers import pipeline, logging as hf_logging
# hf_logging.set_verbosity_error()

# ch1. 허깅페이스

* Inference API 이용 : 모델의 결과를 surver에서
* pipeline() 이용 : 모델을 다운로드받아 모델의 결과를 local에서
    * raw text -> tokenizer -> model -> [0.11, 0.55, 0.xx, ~] logits값으로 prediction 결과 출력
```
허깅페이스 transformers에서 지원하는 task
"sentiment-analysis" : "text-classification"의 별칭(감정분석 전용으로 사용)
"text-classification" : 감정분석, 뉴스분류, 리뷰 분류 등 일반적인 문장 분류
"zero-shot-classification" : 레이블을 학습 없이 주어진 후보군 중에서 분류
"token-classification" : 개체명 인식(NER ; Named Entity REcognition) 등 단위 라벨링
"ner" : "token-classification"의 별칭
"fill-mask" : 빈칸 채우기
"text-generation" : 텍스트 생성 (GPT류 모델에 사용)
"text2text-generation" : 번역, 요약 등 입력 -> 출력 변환 
"translation" : 번역
"summarization" : 텍스트요약
"question-answering" : 주어진 context를 보고 질문에 답하기.
"image-to-text" : 그림을 설명
"image-classification" : 이미지분류
```

# 1. 텍스트 기반 감정분석(긍정/부정)

In [3]:
from transformers import pipeline
classifier = pipeline(task="sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228}]

In [4]:
from transformers import pipeline
classifier = pipeline(task="text-classification",
                     model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
# 감정분석시 내용이 많으면 list로
classifier([
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
])


Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [5]:
[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [6]:
[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [7]:
classifier("이 물건 정말 사고 싶어요")

[{'label': 'POSITIVE', 'score': 0.8577604293823242}]

In [8]:
classifier(["I like you", "I hat you", "나 너가 싫어", "힘들어요"])

[{'label': 'POSITIVE', 'score': 0.9998695850372314},
 {'label': 'POSITIVE', 'score': 0.999488353729248},
 {'label': 'NEGATIVE', 'score': 0.599323034286499},
 {'label': 'POSITIVE', 'score': 0.8669533729553223}]

In [12]:
from transformers import pipeline
classifier = pipeline(task="sentiment-analysis",
                     model="matthewburke/korean_sentiment")
texts = ['나는 너가 좋아', "당신이 싫어요", "힘들어요", "오늘 기분이 최고야"]
result = classifier(texts)

Device set to use cpu


In [5]:
for text, result in zip(texts, classifier(texts)):
    label = "긍정" if result['label']=='LABEL_1' else "부정"
    print(f"{text} => {label} : {result['score']:.4f}")

NameError: name 'texts' is not defined

# 2. 제로샷분류(Zero-shot분류)

* 기계학습 및 자연어처리에서 각 개별 작업에 대한 특정 교육없이 작업을 수행할 수 있는 모형(비지도학습)

In [7]:
classifier = pipeline("zero-shot-classification",
                     # model="facebook/bart-large-mnli"
                     )
classifier(
    "I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"]
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json: 0.00B [00:00, ?B/s]

C:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5227580070495605,
  0.45814019441604614,
  0.0142647260800004,
  0.0026850001886487007,
  0.002152054337784648]}

In [8]:
sequence_to_classify = "One day I well see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'One day I well see the world',
 'labels': ['travel', 'cooking', 'dancing'],
 'scores': [0.9938077926635742, 0.003099897177889943, 0.003092351369559765]}

# 3. text 생성

In [9]:
from transformers import pipeline, set_seed
# set_seed(2)
generation = pipeline("text-generation", "gpt2") # 텍스트 생성 gpt3부터는 허깅페이스없음
generation(
    "in this course. We will teach you how to",
    pad_token_id=generation.tokenizer.eos_token_id
) # pad_token_id 경고를 없애려고 setting

Device set to use cpu


[{'generated_text': 'in this course. We will teach you how to use the tools of the trade to gain valuable insight into the trade process. This course is designed to teach you how to build a solid and professional resume. We will also teach you how to build a strong business and to help you develop a solid and professional resume. You will learn to work with people who are knowledgeable in the trade, and you will learn how to build your resume. We will also teach you how to build a strong team that will succeed in the future and will be a force in your future. This course is designed to teach you how to build a solid and professional resume. We will also teach you how to build a strong team that will succeed in the future and will be a force in your future.\n\nCourse Overview\n\nWe will show you how to build a strong resume with a solid, professional resume. We will also show you how to build a strong resume with a solid, professional resume. We will also show you how to build a strong 

In [10]:
result = generation(
    "in this course. We will teach you how to",
    pad_token_id=generation.tokenizer.eos_token_id
) 
print(result[0]['generated_text'])

in this course. We will teach you how to create and use CSS3 widgets for your website, and how to build responsive pages and responsive CSS pages inside of your website.

The Course

The course is taught in our Introduction to CSS3. It covers using our CSS3 widgets system, and using a variety of technologies to create responsive websites in JavaScript and CSS.

The course is also divided into 5 sections which are divided into 3 sections, which will cover our core concepts:

Introduction to CSS3 widgets

Introduction to HTML 5-based CSS widgets

How to create and use responsive pages and responsive CSS pages inside of your website

How to build responsive pages and responsive CSS pages inside of your website

How to build responsive pages and responsive CSS pages inside of your website

How to build responsive pages and responsive CSS pages inside of your website

How to build responsive pages and responsive CSS pages inside of your website

How to build responsive pages and responsive 

In [11]:
from transformers import pipeline
generation = pipeline("text-generation", "skt/kogpt2-base-v2")
result = generation(
    "이 과정은 다음과 같은 방법을 알려드려요. ",
    pad_token_id = generation.tokenizer.eos_token_id,
    max_new_tokens = 200, # 생성할 최대 길이(생성할 토큰 수)
#     num_return_sequences=1,    # 생성할 문장 갯수
#     do_sample=True,            # 다양한 샘플 사용
#     top_k=50,            # top-k 샘플링(확률 높은 상위 50개 토큰만 사용)         
#     top_p=0.95,          # 확률이 높은 순서대로 95%가 될 때까지의 단어들로만 후보로 사용
#     temperature=1.0,     # 창의성 조절(낮을 수록 보수적)
#     no_repeat_ngram_size=2 # 반복 방지
)
print(result[0]['generated_text'])

config.json: 0.00B [00:00, ?B/s]

C:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--skt--kogpt2-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


이 과정은 다음과 같은 방법을 알려드려요. 죠셉이라는 자가 말입니다."
"알겠습니다. 당신처럼 당신을 사랑하는 사람은 당신이 만든 것이 아니라 당신이 만든 것입니다."
"당신 같은 사람은 당신이 만든 것이 아니라 당신이 만든 것입니다. 당신이 만든 것이 아니라 당신이 만든 것입니다."
당신은 다른 방법을 생각해내지 못했다.
당신의 생각에는 그런 것이 없었다.
"당신이 만든 것이라면 당신이 원하는 것은 무엇일까?"
"좋습니다. 당신은 당신을 사랑합니다."
그러자 한 사람이 대답했다.
"당신을 사랑합니다, 당신. 당신을 사랑합니다."
당신은 고개를 흔들며 말했다.
"당신은 당신이 만든 것입니다. 당신은 당신을 사랑합니다. 당신은 당신을 사랑합니다."
"당신은 당신을 사랑합니다."
그러자 한 사람이 말했다.
"저는 당신이 만든 것입니다. 당신은 당신을 사랑합니다."
"그럼 당신은 당신을 사랑합니까?"
"당신은 당신을 사랑합니다."
"당신은 당신을 사랑합니다."
그런 다음 두 사람을 불러서 이야기를 나누어 들었다.
"당


# 4. 마스크(빈칸) 채우기

In [12]:
unmasker = pipeline(task='fill-mask',
                   model='distilbert/distilroberta-base') # 마스크 채우기
unmasker("I'm going to hospital and meet a <mask>", top_k=2) # top_k 기본값 5

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

C:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--distilbert--distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'score': 0.19275707006454468,
  'token': 3299,
  'token_str': ' doctor',
  'sequence': "I'm going to hospital and meet a doctor"},
 {'score': 0.06794589757919312,
  'token': 27321,
  'token_str': ' psychiatrist',
  'sequence': "I'm going to hospital and meet a psychiatrist"}]

In [13]:
#unmasker("병원에 가서 <mask>를 만날 거예요")

In [14]:
unmasker("Hello, I'm a <mask> model.")

[{'score': 0.0629730075597763,
  'token': 265,
  'token_str': ' business',
  'sequence': "Hello, I'm a business model."},
 {'score': 0.038101598620414734,
  'token': 18150,
  'token_str': ' freelance',
  'sequence': "Hello, I'm a freelance model."},
 {'score': 0.03764132782816887,
  'token': 774,
  'token_str': ' role',
  'sequence': "Hello, I'm a role model."},
 {'score': 0.037326786667108536,
  'token': 2734,
  'token_str': ' fashion',
  'sequence': "Hello, I'm a fashion model."},
 {'score': 0.026023676618933678,
  'token': 24526,
  'token_str': ' Playboy',
  'sequence': "Hello, I'm a Playboy model."}]

In [15]:
unmasker("안녕하세요? 나는 <mask> 모델이예요.", top_k=3)

[{'score': 0.14130638539791107,
  'token': 35,
  'token_str': ':',
  'sequence': '안녕하세요? 나는: 모델이예요.'},
 {'score': 0.1223798543214798,
  'token': 116,
  'token_str': '?',
  'sequence': '안녕하세요? 나는? 모델이예요.'},
 {'score': 0.08188082277774811,
  'token': 328,
  'token_str': '!',
  'sequence': '안녕하세요? 나는! 모델이예요.'}]

###  InferenceAPI

In [20]:
from dotenv import load_dotenv
import os
load_dotenv()
# os.environ['HF_TOKEN']

True